In [ ]:
import warnings
import matplotlib.pyplot as plt
import os
import sys
import time
import pandas as pd
import tkinter as tk
import tkinter.filedialog as fd
from tqdm.notebook import tqdm_notebook
from datetime import datetime
import seaborn as sns
# from multiprocessing import Pool
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# import ray 
# ray.init(num_gpus=0,ignore_reinit_error=True)
# import modin.pandas as mpd
# import numpy as np

In [ ]:
#Enabling notebook extension jupyter-js-widgets/extension...
!jupyter nbextension enable --py widgetsnbextension

# Hide these warnings
warnings.filterwarnings(action='once', message='.*Mean of empty slice.*')

# Some plotting presets
plt.rc("figure", dpi=70)
plt.rc("savefig", dpi=30, facecolor="white", bbox="tight")

In [ ]:
#print python version
python_version = sys.version
print(python_version)

## Choose analysis files
initial_dir = "C:\\Users"
print("Please select the files to analyze:")

time.sleep(0.25)
root = tk.Tk()
root.filenames = fd.askopenfilenames(initialdir=initial_dir,
                                     title="Please select the files to analyze",
                                     filetypes=(("csv files", "*.csv"), ("all files", "*.*")))
input_files = [os.path.basename(f) for f in root.filenames]
input_dir = os.path.dirname(root.filenames[0])
root.destroy()

print("Please choose the output folder:")
time.sleep(0.25)

root = tk.Tk()
root.filenames = fd.askdirectory(initialdir=input_dir,
                                 title="Please select the output folder")
output_dir = root.filenames
root.destroy()

os.chdir(input_dir)
print('Working directory is:', input_dir, '\n')
print('Output folder is:', output_dir,'\n')

In [ ]:
# Initialize a dictionary containing the original and excel-friendly names of all the per-well metrics desired
info_per_day = {
    'Original File Time':'Original File Time',
    'Experiment Start Time':'Experiment Start Time',
    'Well':'Well', # Encabezado de datos por pocillo
    'Active':'Active',
    'Well Coloring':'Well Coloring',
    'Control':'Control',
    'Treatment':'Treatment',
    'Concentration':'Concentration',
    'Additional Information':'Additional Information',
    'Analysis Start (s):':'Analysis Start (s):',
    'Analysis Duration (s):':'Analysis Duration (s)',
    'Treatment Averages':'Treatment Averages', # Encabezado de datos por tratamiento
    'Measurement':'Electrode'# Encabezado de datos por electrodo
}
# Check if all names are <=31 chars long
for i in info_per_day.values():
    if len(i) > 31:
        print(f'\n{i} in "info_per_day" is {len(i)} characters long, {len(i) - 31} characters over the excel worksheet name limit')
print(f'\n{len(info_per_day)} well info parameters will be extracted from the advanced metrics.csv files')

metrics_per_treatment = {
    'Total Wells':'Total Wells',
    'Number of Spikes - Avg':'Number of Spikes - Avg',
    'Number of Spikes - Std':'Number of Spikes - Std',
    'Mean Firing Rate (Hz) - Avg':'Mean Firing Rate (Hz) - Avg',
    'Mean Firing Rate (Hz) - Std':'Mean Firing Rate (Hz) - Std',
    'Number of Active Electrodes - Avg':'NumberActiveElect-Avg',
    'Number of Active Electrodes - Std':'NumberActiveElect-Std',
    'Weighted Mean Firing Rate (Hz) - Avg':'WeightedSpikeRate(Hz)-Avg',
    'Weighted Mean Firing Rate (Hz) - Std':'WeightedSpikeRate(Hz)-Std',
    'Number of Bursts - Avg':'Number of Bursts - Avg',
    'Number of Bursts - Std':'Number of Bursts - Std',
    'Burst Duration - Avg (s)':'Burst Duration - Avg (s)',
    'Burst Duration - Std (s)':'Burst Duration - Std (s)',
    'Inter-Burst Interval - Avg (s)':'Inter-Burst Interval - Avg (s)',
    'Inter-Burst Interval - Std (s)':'Inter-Burst Interval - Std (s)',
    'Burst Frequency - Avg (Hz)':'Burst Frequency - Avg (Hz)',
    'Burst Frequency - Std (Hz)':'Burst Frequency - Std (Hz)',
    'Normalized Duration IQR - Avg':'Normalized Duration IQR - Avg',
    'Normalized Duration IQR - Std':'Normalized Duration IQR - Std',
    'IBI Coefficient of Variation - Avg':'IBICoeffVar-Avg',
    'IBI Coefficient of Variation - Std':'IBICoeffVar-Std',
    'Burst Percentage - Avg':'Burst Percentage - Avg',
    'Burst Percentage - Std':'Burst Percentage - Std',
    'Number of Network Bursts - Avg':'Number of Network Bursts - Avg',
    'Number of Network Bursts - Std':'Number of Network Bursts - Std',
    'Network Burst Frequency - Avg (Hz)':'NetBurst Freq-Avg(Hz)',
    'Network Burst Frequency - Std (Hz)':'NetBurstFreq-Std(Hz)',
    'Network Burst Duration - Avg (sec)':'NetBurstDuration-Avg(sec)',
    'Network Burst Duration - Std (sec)':'NetBurstDuration-Std(sec)',
    'Number of Elecs Participating in Burst - Avg':'ElecsParticipatingInBurst-Avg',
    'Number of Elecs Participating in Burst - Std':'ElecsParticipatingInBurst-Std',
    'Network Burst Percentage - Avg':'Network Burst Percentage - Avg',
    'Network Burst Percentage - Std':'Network Burst Percentage - Std',
    'Network IBI Coefficient of Variation - Avg':'NetIBICoeffVariation-Avg',
    'Network IBI Coefficient of Variation - Std':'NetIBICoeffVariation-Std',
    'Network Normalized Duration IQR - Avg':'NetNormalizedDurationIQR-Avg',
    'Network Normalized Duration IQR - Std':'NetNormalizedDurationIQR-Std',
    'Area Under Normalized Cross-Correlation - Avg':'AreaUnderNormCross-Corr-Avg',
    'Area Under Normalized Cross-Correlation - Std':'AreaUnderNormCross-Corr-Std',
    'Area Under Cross-Correlation - Avg':'AreaUnderCross-Correlation-Avg',
    'Area Under Cross-Correlation - Std':'AreaUnderCross-Correlation-Std'
}

# Check if all names are <=31 chars long
for i in metrics_per_treatment.values():
    if len(i) > 31:
        print(f'\n{i} in "metrics_per_treatment" is {len(i)} characters long, {len(i) - 31} characters over the excel worksheet name limit')
print(f'\n{len(metrics_per_treatment)} treatment metrics will be extracted from the advanced metrics.csv files')

metrics_per_well = {
    'Number of Spikes':'Number_of_Spikes',
    'Mean Firing Rate (Hz)':'Mean_Firing_Rate_Hz_well',
    'Number of Active Electrodes':'Number_of_Active_Electrodes',
    'Weighted Mean Firing Rate (Hz)':'Weighted_Mean_Firing_Rate_Hz',
    'ISI Coefficient of Variation - Avg':'ISI_Coeff_of_Variation_Avg',
    'Number of Bursts':'Number_of_Bursts',
    'Number of Bursting Electrodes':'Number_of_Bursting_Electrodes',
    'Burst Duration - Avg (s)':'Burst_Duration_Avg_s',
    'Burst Duration - Std (s)':'Burst_Duration_Std_s',
    'Number of Spikes per Burst - Avg':'Spikes_per_Burst_Avg',
    'Number of Spikes per Burst - Std':'Spikes_per_Burst_Std',
    'Mean ISI within Burst - Avg':'Mean_ISI_within_Burst_Avg',
    'Mean ISI within Burst - Std':'Mean_ISI_within_Burst_Std',
    'Median ISI within Burst - Avg':'Median_ISI_within_Burst_Avg',
    'Median ISI within Burst - Std':'Median_ISI_within_Burst_Std',
    'Inter-Burst Interval - Avg (s)':'Inter_Burst_Interval_Avg_s',
    'Inter-Burst Interval - Std (s)':'Inter_Burst_Interval_Std_s',
    'Burst Frequency - Avg (Hz)':'Burst_Frequency_Avg_Hz',
    'Burst Frequency - Std (Hz)':'Burst_Frequency_Std_Hz',
    'Normalized Duration IQR - Avg':'Normalized_Duration_IQR_Avg',
    'Normalized Duration IQR - Std':'Normalized_Duration_IQR_Std',
    'IBI Coefficient of Variation - Avg':'IBI_Coeff_of_Variation_Avg',
    'IBI Coefficient of Variation - Std':'IBI_Coeff_of_Variation_Std',
    'Burst Percentage - Avg':'Burst_Percentage_Avg',
    'Burst Percentage - Std':'Burst_Percentage_Std',
    'Number of Network Bursts':'Number_of_Network_Bursts',
    'Network Burst Frequency (Hz)':'Network_Burst_Frequency_Hz',
    'Network Burst Duration - Avg (sec)':'Network_Burst_Duration_Avg_s',
    'Network Burst Duration - Std (sec)':'Network_Burst_Duration_Std_s',
    'Number of Spikes per Network Burst - Avg':'Spikes_per_Netw_Burst_Avg',
    'Number of Spikes per Network Burst - Std':'Spikes_per_Netw_Burst_Std',
    'Number of Elecs Participating in Burst - Avg':'Elecs_Particip_in_Brst_Avg',
    'Number of Elecs Participating in Burst - Std':'Elecs_Particip_in_Brst_Std',
    'Number of Spikes per Network Burst per Channel - Avg':'Spikes_Net_Brst_Channel_Avg',
    'Number of Spikes per Network Burst per Channel - Std':'Spikes_Net_Brst_Channel_Std',
    'Network Burst Percentage':'Network_Burst_Percentage',
    'Network IBI Coefficient of Variation':'Network_IBI_Coeff_Variation',
    'Network ISI Coefficient of Variation':'Network_ISI_Coeff_Variation',
    'Network Normalized Duration IQR':'Network_Norm_Duration_IQR',
    'Area Under Normalized Cross-Correlation':'Area_Under_Norm_Cross_Corr',
    'Area Under Cross-Correlation':'Area_Under_Cross_Correlation',
    'Width at Half Height of Normalized Cross-Correlation':'Width_Hf_Height_N_Cross_Corr',
    'Width at Half Height of Cross-Correlation':'Width_Hf_Height_Cross_Corr',
    'Synchrony Index':'Synchrony_Index'
    }

# Check all names are <=31 chars long
for i in metrics_per_well.values():
    if len(i) > 31:
        print(f'\n{i} in "metrics_per_well" is {len(i)} characters long, {len(i) - 31} characters over the excel worksheet name limit')
print(f'\n{len(metrics_per_well)} well metrics will be extracted from the advanced metrics.csv files')

# Initialize a dictionary containing the original and excel-friendly names of all the per-electrode metrics desired
metrics_per_electrode = {
    'Number of Spikes':'Number_of_Spikes',
    'Mean Firing Rate (Hz)':'Mean_Firing_Rate_Hz',
    'ISI Coefficient of Variation':'ISI_Coefficient_of_Variation',
    'Number of Bursts':'Number_of_Bursts',
    'Burst Duration - Avg (s)':'Burst_Duration_Avg_s',
    'Burst Duration - Std (s)':'Burst_Duration_Std_s',
    'Number of Spikes per Burst - Avg':'Spikes_per_Burst_Avg',
    'Number of Spikes per Burst - Std':'Spikes_per_Burst_Std',
    'Mean ISI within Burst - Avg':'Mean_ISI_within_Burst_Avg',
    'Mean ISI within Burst - Std':'Mean_ISI_within_Burst_Std',
    'Median ISI within Burst - Avg':'Median_ISI_within_Burst_Avg',
    'Median ISI within Burst - Std':'Median_ISI_within_Burst_Std',
    'Inter-Burst Interval - Avg (s)':'Inter_Burst_Interval_Avg_s',
    'Inter-Burst Interval - Std (s)':'Inter_Burst_Interval_Std_s',
    'Burst Frequency (Hz)':'Burst_Frequency_Hz',
    'IBI Coefficient of Variation':'IBI_Coefficient_of_Variation',
    'Normalized Duration IQR':'Normalized Duration IQR',
    'Burst Percentage':'Burst_Percentage',
}

# Check all names are <=31 chars long
for i in metrics_per_electrode.values():
    if len(i) > 31:
        print(f'\n{i} in "metrics_per_electrode" is {len(i)} characters long, {len(i) - 31} characters over the excel worksheet name limit')
print(f'\n{len(metrics_per_electrode)} electrode metrics will be extracted from the advanced metrics.csv files')

In [ ]:
# Before starting with the DataFrames, set display options to print all rows and columns in the notebook
pd.set_option('display.min_rows', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Load all data from the selected input advanced_metrics files as pandas dataframes
well_and_electrode_info = {}
filtered_dict_advanced_metrics_files_treatment = {}
filtered_dict_advanced_metrics_files_well = {}
filtered_dict_advanced_metrics_files_electrode = {}
advanced_metrics_files = [fl for fl in input_files if '_spike_list' not in fl]
spike_files = [fl for fl in input_files if '_spike_list' in fl]

for fl in tqdm_notebook(advanced_metrics_files,
                        desc = f'Retrieving advanced metrics.csv data on {str(time.ctime(time.time()))}'):
        temp = pd.read_csv(fl,
                           header=None,sep='\t',
                           skip_blank_lines=False,
                           engine='python')[0].str.strip().str.split(',', expand=True).set_index([0])
        well_and_electrode_info[fl] =  temp[temp.index.isin(info_per_day.keys())] # Add to the dictionary only those rows from the csv file
                                                                                                # whose index name is among the info_per_day dict keys
        
        filtered_dict_advanced_metrics_files_treatment[fl] = temp.loc['Treatment Averages':'Area Under Cross-Correlation - Std']\
                    [temp.loc['Treatment Averages':'Area Under Cross-Correlation - Std'].index.isin(metrics_per_treatment.keys())]\
                    .replace('', '0', inplace=False).dropna(how='all', axis=1)  # Add indexed rows from the "fl" csv file
            
        filtered_dict_advanced_metrics_files_well[fl] = temp.loc['Well Averages':'Synchrony Index']\
                    [temp.loc['Well Averages':'Synchrony Index'].index.isin(metrics_per_well.keys())]\
                    .replace('', '0', inplace=False).dropna(how='all', axis=1) # Add indexed rows from the "fl" csv file
        
        filtered_dict_advanced_metrics_files_electrode[fl] = temp.loc['Measurement':]\
                    [temp.loc['Measurement':].index.isin(metrics_per_electrode.keys())]\
                    .replace('', '0', inplace=False).dropna(how='all', axis=1) # Add indexed rows from the "fl" csv file

In [ ]:
# Load all data from the selected input _spike_ files as pandas dataframes
dict_spike_files = {}
for fl in tqdm_notebook(spike_files,
                        desc = f'Retrieving spike_file.csv data on {str(time.ctime(time.time()))}'):
    dict_spike_files[fl] = pd.read_csv(fl,
               header=None,sep='\t',
               skip_blank_lines=False,
               engine='python')[0].str.split(',', expand=True) # .fillna(0)
    dict_spike_files[fl].columns = dict_spike_files[fl].iloc[0]
    dict_spike_files[fl] = dict_spike_files[fl].iloc[1:].reset_index(drop=True).rename(columns={'Investigator': 'Information (keys)','':'Information (values)'}).set_index(['Information (keys)'])

In [ ]:
# Read one example of the just-created well_and_electrode_info pandas dataframes with Mito
well_and_electrode_info[advanced_metrics_files[0]]

# Read one example of the just-created filtered_advanced_metrics_treatment pandas dataframes with Mito
filtered_dict_advanced_metrics_files_treatment[advanced_metrics_files[0]]

# Read one example of the just-created filtered_advanced_metrics_well pandas dataframes with Mito
filtered_dict_advanced_metrics_files_well[advanced_metrics_files[0]]

# Read one example of the just-created filtered_advanced_metrics_electrode pandas dataframes with Mito
filtered_dict_advanced_metrics_files_electrode[advanced_metrics_files[0]]

# Read one example of the just-created spike_files pandas dataframes with Mito
dict_spike_files[spike_files[0]].head()

In [ ]:
# Sort data dictionaries by the value of their "Original file Time" row in the well_and_electrode_info dict
well_and_electrode_info = dict(sorted(well_and_electrode_info.items(), key=lambda x: datetime.strptime(x[1].loc['Original File Time'][1], '%m/%d/%Y %H:%M:%S')))
filtered_dict_advanced_metrics_files_treatment = dict([(time_point, filtered_dict_advanced_metrics_files_treatment[time_point]) for time_point in well_and_electrode_info.keys()])
filtered_dict_advanced_metrics_files_well = dict([(time_point, filtered_dict_advanced_metrics_files_well[time_point]) for time_point in well_and_electrode_info.keys()])
filtered_dict_advanced_metrics_files_electrode = dict([(time_point, filtered_dict_advanced_metrics_files_electrode[time_point]) for time_point in well_and_electrode_info.keys()])
dict_spike_files = dict([(f"{time_point.split('.csv')[0]}{'_spike_list.csv'}", dict_spike_files[f"{time_point.split('.csv')[0]}{'_spike_list.csv'}"]) for time_point in well_and_electrode_info.keys()])

In [ ]:
# Create spike amplitude dataframes per treatment, per well and per electrode in same structure as advanced metrics
spikes_per_day_tables = {'Treatment':{},'Well':{},'Electrode':{}}
baseline_day = [day for day in dict_spike_files.keys() if 'bl' in day][0]
baseline_amps = dict_spike_files[baseline_day]

In [ ]:
# per treatment

headers_treat = [
# Headers are: treatment and coloring, in that order
list(well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Treatment Averages'].dropna()),
[{t: c for t, c in zip(well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Treatment'].dropna(),well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Well Coloring'].dropna())}[x]
 for x in list(well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Treatment Averages'].dropna())],
]
tuples_treat = list(zip(*headers_treat))
index_treat = pd.MultiIndex.from_tuples(tuples_treat, names=['Treatment Averages', 'Coloring'])

spikes_per_day_tables['Treatment']['Amplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_treat)
spikes_per_day_tables['Treatment']['nAmplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_treat)
for day, ampdata in tqdm_notebook(dict_spike_files.items(),
                                 desc = f'Filling amplitude dataframes on {str(time.ctime(time.time()))}'):
    wells_treat = {t:[] for w,t in zip(ampdata.loc['Well'],ampdata.loc['Treatment']) if t == t}
    for w,t in zip(ampdata.loc['Well'],ampdata.loc['Treatment']):
        wells_treat[t].append(w)
    for treatment in tqdm_notebook(headers_treat[0],
                                  desc = f'Filling amplitude dataframes for {day} on {str(time.ctime(time.time()))}'):
        spikes_per_day_tables['Treatment']['Amplitude (uV)'].loc[day,treatment] = ampdata[:-9].query('Electrode.str.slice(0,2) in @wells_treat[@treatment]',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000
        with pd.option_context('mode.use_inf_as_na', True): # this option prevents black color displaying for infinite values by background_gradient    
            spikes_per_day_tables['Treatment']['nAmplitude (uV)'].loc[day,treatment] = (ampdata[:-9].query('Electrode.str.slice(0,2) in @wells_treat[@treatment]',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000)/(baseline_amps[:-9].query('Electrode.str.slice(0,2) in @wells_treat[@treatment]')['Amplitude (mV)'].astype(float).mean()*1000)

In [ ]:
# per well

headers_well = [
# Headers are: well,treatment and coloring, in that order
list(dict_spike_files[spike_files[-1]].loc['Well'].dropna()),
list(dict_spike_files[spike_files[-1]].loc['Treatment'].dropna()),
list(dict_spike_files[spike_files[-1]].loc['Well Coloring'].dropna())
]
tuples_well = list(zip(*headers_well))
index_well = pd.MultiIndex.from_tuples(tuples_well, names=['Well', 'Treatment', 'Coloring'])

spikes_per_day_tables['Well']['Amplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_well)
spikes_per_day_tables['Well']['nAmplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_well)
for day, ampdata in tqdm_notebook(dict_spike_files.items(),
                                 desc = f'Filling amplitude dataframes on {str(time.ctime(time.time()))}'):
    for well in tqdm_notebook(headers_well[0],
                                  desc = f'Filling amplitude dataframes for {day} on {str(time.ctime(time.time()))}'):
        spikes_per_day_tables['Well']['Amplitude (uV)'].loc[day,well] = ampdata[:-9].query('Electrode.str.slice(0,2) == @well',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000
        with pd.option_context('mode.use_inf_as_na', True): # this option prevents black color displaying for infinite values by background_gradient    
            spikes_per_day_tables['Well']['nAmplitude (uV)'].loc[day,well] = (ampdata[:-9].query('Electrode.str.slice(0,2) == @well',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000)/(baseline_amps[:-9].query('Electrode.str.slice(0,2) == @well')['Amplitude (mV)'].astype(float).mean()*1000)

In [ ]:
# per electrode
# Alternative is to only run this cell and produce the per-well and per-treatment using parallelized pd.groupby()
headers_elec = [
# Headers are: electrode (measurement), treatment and coloring, in that order
list(well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Measurement'].dropna()),
list(w for t, w in zip(dict_spike_files[spike_files[-1]].loc['Treatment'],
dict_spike_files[spike_files[-1]].loc['Well']) for e in
well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Measurement'].dropna() if e[:2] == w),
list(t for t, w in zip(dict_spike_files[spike_files[-1]].loc['Treatment'],
dict_spike_files[spike_files[-1]].loc['Well']) for e in
well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Measurement'].dropna() if e[:2] == w),
list(c for c, w in zip(dict_spike_files[spike_files[-1]].loc['Well Coloring'],
dict_spike_files[spike_files[-1]].loc['Well']) for e in
well_and_electrode_info[spike_files[-1].replace('_spike_list','')].loc['Measurement'].dropna() if e[:2] == w)
]
tuples_elec = list(zip(*headers_elec))
index_elec = pd.MultiIndex.from_tuples(tuples_elec, names=['Electrode', 'Well', 'Treatment', 'Coloring'])

spikes_per_day_tables['Electrode']['Amplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_elec)
spikes_per_day_tables['Electrode']['nAmplitude (uV)'] = pd.DataFrame(index=dict_spike_files.keys(),
                            columns=index_elec)
for day, ampdata in tqdm_notebook(dict_spike_files.items(),
                                 desc = f'Filling amplitude dataframes on {str(time.ctime(time.time()))}'):
    for electrode in tqdm_notebook(headers_elec[0],
                                  desc = f'Filling amplitude dataframes for {day} on {str(time.ctime(time.time()))}'):
        spikes_per_day_tables['Electrode']['Amplitude (uV)'].loc[day,electrode] = ampdata.query('Electrode == @electrode',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000
        with pd.option_context('mode.use_inf_as_na', True): # this option prevents black color displaying for infinite values by background_gradient    
            spikes_per_day_tables['Electrode']['nAmplitude (uV)'].loc[day,electrode] = (ampdata.query('Electrode == @electrode',inplace=False)['Amplitude (mV)'].astype(float).mean()*1000)/(baseline_amps.query('Electrode == @electrode')['Amplitude (mV)'].astype(float).mean()*1000)

In [ ]:
# Create a dictionary of DataFrames, each containing the data for each metric, organized by day (rows) and TREATMENT (columns)

metrics_per_day_and_treatment_tables = {}
metrics_per_day_and_treatment_tables['Amplitude (uV)'] = spikes_per_day_tables['Treatment']['Amplitude (uV)'].fillna(0)
metrics_per_day_and_treatment_tables['nAmplitude (uV)'] = spikes_per_day_tables['Treatment']['nAmplitude (uV)'].fillna(0)
baseline_day = [day for day in filtered_dict_advanced_metrics_files_treatment.keys() if 'bl' in day][0]
baseline_df = filtered_dict_advanced_metrics_files_treatment[baseline_day]

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_treatment.items(),
                                    desc = f'Initializing per-treatment dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_treatment.items():
        headers_treatment = [
        # Headers are: treatment and coloring, in that order
        list(well_and_electrode_info[day].loc['Treatment Averages'].dropna()),
        [{t: c for t, c in zip(well_and_electrode_info[day].loc['Treatment'].dropna(),well_and_electrode_info[day].loc['Well Coloring'].dropna())}[x]
         for x in list(well_and_electrode_info[day].loc['Treatment Averages'].dropna())],
        ]
        tuples_treat = list(zip(*headers_treatment))
        index_treat = pd.MultiIndex.from_tuples(tuples_treat, names=['Treatment Averages', 'Coloring'])
        metrics_per_day_and_treatment_tables[v] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_treatment.keys(),
                                columns=index_treat)
        metrics_per_day_and_treatment_tables[f'n{v}'] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_treatment.keys(),
                                columns=index_treat)

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_treatment.items(),
                                    desc = f'Filling per-treatment dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_treatment.items():
        metrics_per_day_and_treatment_tables[v].loc[day] = list(dataframe.loc[k])
        with pd.option_context('mode.use_inf_as_na', True): # this option prevents black color displaying for infinite values by background_gradient 
            metrics_per_day_and_treatment_tables[f'n{v}'].loc[day] = list((dataframe.loc[k].astype(float)/baseline_df.loc[k].astype(float)).fillna(0))
        
with pd.ExcelWriter('Adv_metrics_treatment.xlsx', engine='xlsxwriter',
                    engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
    for df_name, df in tqdm_notebook(metrics_per_day_and_treatment_tables.items(),
                                    desc = f'Writing per-treatment dictionaries to excel on {str(time.ctime(time.time()))}'):
        styler = df.sort_values(by=['Treatment Averages'],axis=1).style 
        for color in set(index_treat.get_level_values('Coloring')):
            styler.background_gradient(cmap=sns.light_palette(color, as_cmap=True),
                                       axis=None,low=0.5,high=0.7,
                                       subset=[w for w, c in zip(list(index_treat.get_level_values('Treatment Averages')),
                                                                 list(index_treat.get_level_values('Coloring'))) if c==color])
        styler.to_excel(writer, sheet_name=df_name)
        for column in df:
            column_width = max(df.index.astype(str).map(len).max(), len(column))
            col_idx = df.columns.get_loc(column)
            writer.sheets[df_name].set_column(col_idx, col_idx, column_width)

In [ ]:
# Create a dictionary of DataFrames containing the data for each metric organized by day (rows) and WELL (columns)

metrics_per_day_and_well_tables = {}
metrics_per_day_and_well_tables['Amplitude (uV)'] = spikes_per_day_tables['Well']['Amplitude (uV)'].fillna(0)
metrics_per_day_and_well_tables['nAmplitude (uV)'] = spikes_per_day_tables['Well']['nAmplitude (uV)'].fillna(0)
baseline_day = [day for day in filtered_dict_advanced_metrics_files_well.keys() if 'bl' in day][0]
baseline_df = filtered_dict_advanced_metrics_files_well[baseline_day]

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_well.items(),
                                    desc = f'Initializing per-well dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_well.items():
        headers_well = [
        # Headers are: well,treatment and coloring, in that order
        list(well_and_electrode_info[day].loc['Well'].dropna()),
        list(well_and_electrode_info[day].loc['Treatment'].dropna()),
        list(well_and_electrode_info[day].loc['Well Coloring'].dropna())
        ]
        tuples_well = list(zip(*headers_well))
        index_well = pd.MultiIndex.from_tuples(tuples_well, names=['Well', 'Treatment', 'Coloring'])
        metrics_per_day_and_well_tables[v] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_well.keys(),
                                columns=index_well)
        metrics_per_day_and_well_tables[f'n{v}'] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_well.keys(),
                                columns=index_well)

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_well.items(),
                                    desc = f'Filling per-well dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_well.items():
        metrics_per_day_and_well_tables[v].loc[day] = list(dataframe.loc[k])
        with pd.option_context('mode.use_inf_as_na', True):
            metrics_per_day_and_well_tables[f'n{v}'].loc[day] = list((dataframe.loc[k].astype(float)/baseline_df.loc[k].astype(float)).fillna(0))

with pd.ExcelWriter('Adv_metrics_well.xlsx', engine='xlsxwriter',
                    engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
    for df_name, df in tqdm_notebook(metrics_per_day_and_well_tables.items(),
                                    desc = f'Writing per-well dictionaries to excel on {str(time.ctime(time.time()))}'):
        styler = df.sort_values(by=['Treatment'],axis=1).style 
        for color in set(index_well.get_level_values('Coloring')):
            styler.background_gradient(cmap=sns.light_palette(color, as_cmap=True),
                                       axis=None,low=0.5,high=0.7,
                                       subset=[w for w, c in zip(list(index_well.get_level_values('Well')), list(index_well.get_level_values('Coloring'))) if c==color])
        styler.to_excel(writer, sheet_name=df_name)
        for column in df:
            column_width = max(df.index.astype(str).map(len).max(), len(column))
            col_idx = df.columns.get_loc(column)
            writer.sheets[df_name].set_column(col_idx, col_idx, column_width)

In [ ]:
# Create a dictionary of DataFrames containing the data for each metric organized by day (rows) and ELECTRODE (columns)

metrics_per_day_and_electrode_tables = {}
metrics_per_day_and_electrode_tables['Amplitude (uV)'] = spikes_per_day_tables['Electrode']['Amplitude (uV)'].fillna(0)
metrics_per_day_and_electrode_tables['nAmplitude (uV)'] = spikes_per_day_tables['Electrode']['nAmplitude (uV)'].fillna(0)
baseline_day = [day for day in filtered_dict_advanced_metrics_files_electrode.keys() if 'bl' in day][0]
baseline_df = filtered_dict_advanced_metrics_files_electrode[baseline_day]

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_electrode.items(),
                                    desc = f'Initializing per-electrode dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_electrode.items():
        headers_elec = [
        # Headers are: electrode, treatment and coloring, in that order
        list(well_and_electrode_info[day].loc['Measurement'].dropna()),
        list(t for t, w in zip(well_and_electrode_info[day].loc['Treatment'].dropna(),
        well_and_electrode_info[day].loc['Well'].dropna()) for e in
        well_and_electrode_info[day].loc['Measurement'].dropna() if e[:2] == w),
        list(c for c, w in zip(well_and_electrode_info[day].loc['Well Coloring'].dropna(),
        well_and_electrode_info[day].loc['Well'].dropna()) for e in
        well_and_electrode_info[day].loc['Measurement'].dropna() if e[:2] == w)
        ]
        tuples_elec = list(zip(*headers_elec))
        index_elec = pd.MultiIndex.from_tuples(tuples_elec, names=['Electrode', 'Treatment', 'Coloring'])
        metrics_per_day_and_electrode_tables[v] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_electrode.keys(),
                                columns=index_elec)
        metrics_per_day_and_electrode_tables[f'n{v}'] = pd.DataFrame(index=filtered_dict_advanced_metrics_files_electrode.keys(),
                                columns=index_elec)

for day, dataframe in tqdm_notebook(filtered_dict_advanced_metrics_files_electrode.items(),
                                    desc = f'Filling per-electrode dictionaries on {str(time.ctime(time.time()))}'):
    for k, v in metrics_per_electrode.items():
        metrics_per_day_and_electrode_tables[v].loc[day] = list(dataframe.loc[k])
        with pd.option_context('mode.use_inf_as_na', True):
            metrics_per_day_and_electrode_tables[f'n{v}'].loc[day] = list((dataframe.loc[k].astype(float)/baseline_df.loc[k].astype(float)).fillna(0))

with pd.ExcelWriter('Adv_metrics_electrode.xlsx', engine='xlsxwriter',
                    engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
    for df_name, df in tqdm_notebook(metrics_per_day_and_electrode_tables.items(),
                                    desc = f'Writing per-electrode dictionaries to excel on {str(time.ctime(time.time()))}'):
        styler = df.sort_values(by=['Treatment'],axis=1).style 
        for color in set(index_elec.get_level_values('Coloring')):
            styler.background_gradient(cmap=sns.light_palette(color, as_cmap=True),
                                       axis=None,low=0.5,high=0.7,
                                       subset=[e for e, c in zip(list(index_elec.get_level_values('Electrode')), list(index_elec.get_level_values('Coloring'))) if c==color])
        styler.to_excel(writer, sheet_name=df_name)
        for column in df:
            column_width = max(df.index.astype(str).map(len).max(), len(column))
            col_idx = df.columns.get_loc(column)
            writer.sheets[df_name].set_column(col_idx, col_idx, column_width)